CONFIGURAÇÃO DE AMBIENTE


In [ ]:
import subprocess
import sys
import pkg_resources

def upgrade_or_install(packages):
    """Verifica se os pacotes estão instalados e os atualiza somente se estiverem desatualizados."""
    for package in packages:
        try:
            dist = pkg_resources.get_distribution(package)  # Obtém informações sobre o pacote instalado
            installed_version = dist.version

            print(f"✅ A {package} já está instalada (versão {installed_version}). \nVerificando atualizações...")

            # Verifica a versão mais recente disponível no PyPI
            latest_version = subprocess.run([
                sys.executable, "-m", "pip", "index", "versions", package
            ], capture_output=True, text=True).stdout

            if latest_version and installed_version not in latest_version:
                print(f"🔄 Uma nova versão de {package} está disponível.\n Atualizando agora... Isso pode demorar alguns minutos!")
                subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", package], check=True)
                print(f"✅ A {package} foi atualizada com sucesso.")
            else:
                print(f"✔️ A {package} já está atualizada.")

        except pkg_resources.DistributionNotFound:
            print(f"⚠️ A {package} não encontrado. \nIniciando a instalação... Isso pode demorar alguns minutos!")
            subprocess.run([sys.executable, "-m", "pip", "install", package], check=True)
            print(f"✅A {package} foi instalada com sucesso.")

# Lista de pacotes necessários
packages = ["yfinance", "seaborn", "matplotlib", "pandas", "numpy"]

# Verificar e instalar/atualizar pacotes
upgrade_or_install(packages)

✅ 
A biblioteca yfinance já está instalada.
🔄 
A yfinance foi atualizada com sucesso!
✅ 
A biblioteca seaborn já está instalada.
🔄 
A seaborn foi atualizada com sucesso!
✅ 
A biblioteca matplotlib já está instalada.
🔄 
A matplotlib foi atualizada com sucesso!
✅ 
A biblioteca pandas já está instalada.
🔄 
A pandas foi atualizada com sucesso!
✅ 
A biblioteca numpy já está instalada.
🔄 
A numpy foi atualizada com sucesso!


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

# Configuração de visualização
sns.set_theme(style="whitegrid")

# Lista de ações corrigida
stocks = [
    "PETR4.SA", "PRIO3.SA",
    "BBAS3.SA", "BBDC3.SA", "BRSR6.SA", "ABCB4.SA", "SANB4.SA",  # BANCOS
    "BBSE3.SA", "CXSE3.SA",
    "MRVE3.SA", "JHSF3.SA", "LAVV3.SA",
    "BRAP4.SA", "VALE3.SA", "CMIN3.SA",  # MINERAÇÃO
    "CMIG4.SA", "TAEE4.SA", "NEOE3.SA", "EQTL3.SA", "ELET3.SA", "CPLE3.SA", # ENERGIA
    "GOAU4.SA", "GGBR3.SA",
    "RANI3.SA", "SUZB3.SA", "CSAN3.SA",
    "WEGE3.SA", "LEVE3.SA",
    "TIMS3.SA",  # BENS INDUSTRIAIS
    "MGLU3.SA"  # VAREJO
]

def show_data(stocks):
    """ Exibe os dados históricos das ações. """
    try:
        data = yf.download(stocks, start="2023-01-01", end="2025-01-01")["Close"]
        print(data)
        print(f'\n\n O estudo envolve {len(stocks)} ações em diferentes setores da economia brasileira.\n')
        return data
    except Exception as e:
        raise ValueError("🚫 Nenhum dado encontrado para as ações fornecidas. Verifique os códigos e/ou período escolhido.")

def normalize_data(data):
    """ Normaliza os preços para comparar desempenhos """
    data = data.dropna(axis=1, how='all')
    return data / data.iloc[0]

def ibovespa():
    """ Obtém os dados históricos do Ibovespa. """
    try:
        bova = yf.download("^BVSP", start="2023-01-01", end="2025-01-01")["Close"]
        return bova
    except Exception as e:
        raise ValueError(f"🚫 Nenhum dado encontrado para o índice Ibovespa. Erro: {e}")

def calc_dy(acoes):
    """ Verifica e imprime os dividendos pagos por um conjunto de ações. """
    print("\n🔍 **Coletando dados sobre o pagamento de dividendos:**\n")
    print("\nAguarde um momento...\n")
    pagam_dy = []
    nao_pagam_dy = []

    for stock in acoes:
        try:
            acao = yf.Ticker(stock)
            dividendos = acao.dividends
            if not dividendos.empty:
                dividendos_periodo = dividendos.loc["2023-01-01":"2025-01-01"]
                total_dividendos = dividendos_periodo.sum()
                if total_dividendos > 0:
                    pagam_dy.append(f"✅ {stock} pagou dividendos: R$ {total_dividendos:.2f}")
                else:
                    nao_pagam_dy.append(f"❌ {stock} não pagou dividendos no período.")
            else:
                nao_pagam_dy.append(f"❌ {stock} não possui histórico de dividendos.")
        except Exception as e:
            print(f"🚫 Erro ao processar {stock}: {e}")
            continue

    print(f"\n\nAÇÕES QUE PAGAM DIVIDENDOS: {len(pagam_dy)}\n")
    for mensagem in pagam_dy:
        print(mensagem)

    print(f"\nAÇÕES QUE NÃO PAGAM DIVIDENDOS: {len(nao_pagam_dy)}\n")
    for mensagem in nao_pagam_dy:
        print(mensagem)

    return pagam_dy, nao_pagam_dy

def calc_beta(acao_data, mercado_data, normalize=True):
    """
    Calcula o beta de uma ação em relação ao mercado e opcionalmente o normaliza entre 0 e 1.

    Args:
        acao_data (pd.Series): Dados de preço de uma única ação
        mercado_data (pd.Series): Dados de preço do índice de mercado
        normalize (bool): Se True, normaliza o beta entre 0 e 1

    Returns:
        float: Valor do beta da ação (normalizado ou não)
    """
    try:
        # Garantir que os dados sejam séries unidimensionais
        acao_data = acao_data.squeeze().dropna()
        mercado_data = mercado_data.squeeze().dropna()

        # Alinhar os índices temporais
        common_index = acao_data.index.intersection(mercado_data.index)

        # Selecionar apenas os dados do período comum
        acao_data = acao_data.loc[common_index]
        mercado_data = mercado_data.loc[common_index]

        # Calcular retornos
        acao_returns = acao_data.pct_change().dropna()
        mercado_returns = mercado_data.pct_change().dropna()

        # Garantir dimensão 1D antes do cálculo
        acao_returns = acao_returns.values.ravel()
        mercado_returns = mercado_returns.values.ravel()

        # Calcular beta usando fórmula precisa
        cov_matrix = np.cov(acao_returns, mercado_returns)
        beta = cov_matrix[0, 1] / cov_matrix[1, 1]

        if normalize:
            # Para normalizar o beta entre 0 e 1, podemos usar a seguinte abordagem:
            # 1. Primeiro, convertemos valores negativos para positivos usando valor absoluto
            # 2. Depois, aplicamos uma função de normalização que mapeia valores para o intervalo [0,1]
            # 3. Usamos uma função sigmóide modificada para comprimir valores extremos

            # Passo 1: Valor absoluto
            abs_beta = abs(beta)

            # Passo 2: Normalização usando função sigmóide
            normalized_beta = 1 / (1 + np.exp(-abs_beta))

            return normalized_beta

        return beta

    except Exception as e:
        print(f"Erro detalhado no cálculo do beta: {e}")
        print(f"Formato dos dados da ação: {acao_data.shape}")
        print(f"Formato dos dados do mercado: {mercado_data.shape}")
        return None

def print_debug():
    """ Função para exibir informações de debug com validação robusta e ordenação por beta normalizado """
    try:
        # Baixar dados das ações
        data = show_data(stocks)

        # Baixar dados do mercado
        mercado = ibovespa()

        # Lista para armazenar resultados
        resultados = []

        print("\nCALCULANDO BETAS DAS AÇÕES EM RELAÇÃO AO ÍNDICE IBOVESPA...")
        for stock in stocks:
            if stock in data.columns:
                # Extrair série específica da ação
                acao_serie = data[stock]

                # Calcular beta normal e normalizado
                beta_normal = calc_beta(acao_serie, mercado, normalize=False)
                beta_normalized = calc_beta(acao_serie, mercado, normalize=True)

                if beta_normal is not None:
                    resultados.append({
                        'acao': stock,
                        'beta_normal': beta_normal,
                        'beta_normalized': beta_normalized
                    })
            else:
                print(f"\n🚫 Dados não disponíveis para {stock}")

        # Ordenar resultados pelo beta normalizado
        resultados_ordenados = sorted(resultados, key=lambda x: x['beta_normal'])
        top15 = resultados_ordenados[:15]
        # Imprimir resultados ordenados
        print("\nATIVOS ORDENADOS EM FUNÇAÕ DO BETA (DO MENOR PARA O MAIOR RISCO SISTEMÁTICO):")
        print("\n{:<15} {:<15} {:<15}".format("AÇÃO", "BETA", "BETA [0-1]"))
        print("-" * 40)
        for resultado in resultados_ordenados:
            print("{:<15} {:<15.4f} {:<15.4f}".format(
                resultado['acao'],
                resultado['beta_normal'],
                resultado['beta_normalized']
            ))
        print("-" * 40)

        print("{:<15}".format(top15['acao']))
    except Exception as e:
        print(f"Erro durante a execução: {e}")

if __name__ == "__main__":
    print_debug()

[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  1 of 1 completed


Ticker       ABCB4.SA   BBAS3.SA   BBDC3.SA   BBSE3.SA   BRAP4.SA   BRSR6.SA  \
Date                                                                           
2023-01-02  15.633925  14.519229  10.935263  27.927063  23.877615   8.014033   
2023-01-03  15.011599  14.248577  10.418287  27.057217  23.701220   7.886828   
2023-01-04  15.062057  14.431927  10.509604  27.191040  23.925722   7.929230   
2023-01-05  15.919861  15.091095  10.849960  27.249588  24.519054   8.115797   
2023-01-06  15.961910  15.300632  11.140512  28.044157  24.607252   8.124279   
...               ...        ...        ...        ...        ...        ...   
2024-12-20  19.477821  24.080000  10.491935  36.529999  16.860001   9.800000   
2024-12-23  19.410723  23.920000  10.326477  36.490002  16.740000   9.710000   
2024-12-26  19.813316  24.200001  10.443271  36.470001  16.719999  10.040000   
2024-12-27  19.889999  24.110001  10.365408  36.279999  16.610001   9.990000   
2024-12-30  19.309999  24.170000  10.355